In [1]:
import pandas as pd

In [8]:
df = pd.read_csv("../data/COMPL_GLOBAL_BL_CLK36M_TSLOG_MMD_04122025-V2_bot_HLRAJ_v3.4.csv")
df.head()

,customVariable1,Database,reason,source,sorting,ong_format_1,ong_format_2,ong_format_combined
0,4.480000e+11,GLOBAL_BLACKLIST,Invalid_phone,AI_AGENT_OSversion,1.0,+,@yourmobile.com,+447999998980@yourmobile.com
1,4.480000e+11,GLOBAL_BLACKLIST,Invalid_phone,AI_AGENT_OSversion,2.0,+,@yourmobile.com,+447999998262@yourmobile.com
2,4.480000e+11,GLOBAL_BLACKLIST,Invalid_phone,AI_AGENT_OSversion,3.0,+,@yourmobile.com,+447999998172@yourmobile.com
3,4.480000e+11,GLOBAL_BLACKLIST,Invalid_phone,AI_AGENT_OSversion,4.0,+,@yourmobile.com,+447999997175@yourmobile.com
4,4.480000e+11,GLOBAL_BLACKLIST,Invalid_phone,AI_AGENT_OSversion,5.0,+,@yourmobile.com,+447999996691@yourmobile.com


In [7]:
df.columns

Index(['Phone', 'Database', 'resoan', 'source', 'sorting', 'ong_format_1',
       'ong_format_2', 'ong_format_combined '],
      dtype='object')

In [76]:
# Remove some volumns
try:
    # df = df.drop(columns=['Traffic Source ID','Flow ID', 'Path ID',	'Lander ID', 'Affiliate Networks and Offers Map', 'Referrer', 'Device Id Type', 'Custom Variable 2', 'Custom Variable 3', 'Custom Variable 4', 'Custom Variable 5', 'Custom Variable 6', 'Custom Variable 7', 'Custom Variable 8', 'Custom Variable 9', 'Custom Variable 10'])
    df = df.drop(columns=[column for column in df.columns.to_list() if column not in ['Click ID', 'IP', 'OS', 'OS Version', 'Visit Timestamp', 'Country Code', 'Device', 'Custom Variable 1']])
except:
    pass

In [77]:
df.count()

IP                   166
Country Code         166
OS                   166
Device               166
OS Version           166
Custom Variable 1    166
Click ID             166
Visit Timestamp      166
dtype: int64

In [78]:
df["OS Version"].unique()

array(['IOS 18.7', 'IOS 26.1', 'IOS 18.6', 'IOS 18.4', 'IOS 18.5',
       'IOS 26.0', 'IOS 17.6', 'IOS 2', 'IOS 1', 'IOS 17.0', 'IOS 17.5',
       'IOS 17.4', 'IOS 18.3', 'Android 8.0', 'IOS 18.2', 'IOS 26.2',
       'IOS 17.3', 'IOS 16.7', 'IOS 18.1', 'IOS 18.0', 'IOS 16.5',
       'IOS 16.6'], dtype=object)

In [79]:
# If OS Version is from this list Windows 7, Windows XP, Linux Ubuntu, Android 7 or lower, iOS 10 or lower, add to blacklist 
# If OS Version is from this list [Windows 10, Windows 11, Android 8, Android 9, Android (generic), Linux] add to monitor list
# If OS Version is from this list [iOS 11 - 26.2, MacOS, Android 10 - 17.1] add to whitelist

In [80]:
def classifyRecord(osVersion):
    [os, version] = osVersion.split(' ') if osVersion.count(' ') == 1 else [osVersion, 'nil']
    majorVersion = int(version.split('.')[0]) if version != 'nil' and version[0].isdigit() else 9999
    # BLACKLIST rules first   
    if os == 'IOS' and majorVersion <= 10:
        return 'BLACKLIST'   
    if os == 'Android' and majorVersion <= 7:
        return 'BLACKLIST'   
    if osVersion in ['Windows 7', 'Windows XP'] or 'Ubuntu' in osVersion:
        return 'BLACKLIST'

    # Monitor rules
    if os == 'Android' and majorVersion in [8, 9, 9999]:
        return 'MONITOR'
    if osVersion in ['Windows 10', 'Windows 11'] or os == 'Linux':
        return 'MONITOR'

    # Whitelist rules
    if os == 'IOS' and 11 <= majorVersion <= 26:
        return 'WHITELIST'
    if 'MacOS' in os or 'Mac' in os:
        return 'WHITELIST'
    if os == 'Android' and 10 <= majorVersion <= 17:
        return 'WHITELIST'

In [81]:
df["category"] = df["OS Version"].apply(classifyRecord)
df["category"].value_counts()

category
WHITELIST    163
BLACKLIST      2
MONITOR        1
Name: count, dtype: int64

In [79]:
df["OS Version"][df["category"] == 'WHITELIST']
# df["OS Version"][df["category"] == 'MONITOR']


67         IOS 17.2
68         IOS 18.6
69     Android 10.0
70     Android 10.0
71         IOS 18.0
           ...     
195    Android 10.0
196    Android 10.0
197    Android 10.0
198    Android 10.0
199    Android 10.0
Name: OS Version, Length: 126, dtype: object